# **Előkészületek**

*Az Open-Source AI Cookbook documentation alapján készült a backend kód:*

https://huggingface.co/learn/cookbook/rag_evaluation


In [2]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
%pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.3 MB/s eta 

In [5]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

#megelőzzük a Pandasban lévő hosszú szövegek levágását
pd.set_option("display.max_colwidth", None)

In [6]:
from huggingface_hub import notebook_login

#bejelentkezünk a HF HUB API tokennel
notebook_login()

# **Adathalmaz betöltése**

### Wikipédiás fájlok betöltése

In [ ]:
ds= datasets.load_dataset("wikipedia", "20220301.simple", split="train")

In [ ]:
ds

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["title"]}) for doc in tqdm(ds)]
# behelyezi a szöveget és a forrásokat a langchainDocument formátumú listába, itt átneveztem az oszlopot title-re

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
#egy szegmentáló algoritmus mely a megadott paraméterek alapján vágja bizonyos részekre a szöveget
#majd ezeket akár külön szeparátorokkal kezeli és különböző részekre bontja


docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

#miután bekalibráltuk a szegmentáló algoritmust, azután végighaladunk az általunk létrehozott dokumentum listán

### Alternatív betöltés - eredeti fájlok

In [7]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")
#betöltjük az adott adathalmazt

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]
# behelyezi a szöveget és a forrásokat a langchainDocument formátumú listába

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
#egy szegmentáló algoritmus mely a megadott paraméterek alapján vágja bizonyos részekre a szöveget
#majd ezeket akár külön szeparátorokkal kezeli és különböző részekre bontja


docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

#miután bekalibráltuk a szegmentáló algoritmust, azután végighaladunk az általunk létrehozott dokumentum listán

  0%|          | 0/2647 [00:00<?, ?it/s]

## Tesztelő LLM Kliens hívás

In [9]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)
#meghívjuk az InferenceClient modult a huggingface könyvtárból, majd beállítjuk paraméterenek a mistarlAI modelljét
#továbbá korlátozzuk a kérés maximális időtúllépését másodpercekben

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]

#létrehozunk egy metódust, melyben egy

call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [10]:
#Tesztelésként nézzük meg mit reagál egy kérdésre és hogyan tud választ adni rá
result =call_llm(llm_client, "What is the meaning of life?")
print(result)

What is the meaning of life?

This is a question that has puzzled philosophers, theologians, and scientists for centuries. It is a question that has been asked in many different ways, and it has been given many different answers.

Some people believe that the meaning of life is to seek happiness and fulfillment. Others believe that it is to serve a higher power or to follow a set of moral principles. Still others believe that the meaning of life is to learn and grow, to experience new things and to become the best version of ourselves that we can be.

Ultimately, the meaning of life is a personal and subjective question. It is up to each individual to decide what gives their life meaning and purpose. For some, this may be a religious or spiritual belief. For others, it may be a career or a hobby. And for others still, it may be a relationship or a family.

No matter what gives your life meaning, it is important to remember that the journey is just as important as the destination. The e

# **RAG rendszer létrehozása**

In [11]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)
]
#itt létrehozunk egy újabb LangchainDocumentet a forrás dataseg-ből (vagy adattömbből), szintén hasonló formátumban

  0%|          | 0/2647 [00:00<?, ?it/s]

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    #Itt létrehozunk egy listát melyben szintént text splitterrel daraboljuk fel a szöveget
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )
    #itt beraakjuk a kapott langchain documentet- a textsplitter-rel való szétvágás utána  listába

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Kiszedni a duplikátumokat, így csak egyszer fog egy dokumentum szerepelni a végső kimenetben
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

## 2.2 Retriever embedding-ek beállítása

In [13]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    # betölteni az embedding modelt és létrehoz egy FAISS indexet a dokumentumokhoz
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"}, #ezt át lehet állítani, ha nem CUDA alapon szeretnénk futtatni
        encode_kwargs={"normalize_embeddings": True},  # minden beágyazás hossza 1-re lesz normalizálva
        #így lehetővé teszi a koszinusz hasonlóság közvetlen számításást, leegyszerűsíti az analízist
    )

    # megnézni azt, hogy az embedding már létezik-e
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )
    #ha nem akkor lefuttatja az alábbi kódot és visszaadja a létrehozott FAISS indexeket
    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

## 2.3 Reader - LLM

RAG Prompt template beállítása

In [14]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

Reader LLM bevezetése, amely elolvassa a dokumentumokat, ahhoz, hogy válaszoljon

In [15]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    huggingfacehub_api_token= '#HF_HUB_KEY',#HF_HUB_KEY
    repo_id=repo_id,
    task="text-generation",#modellkalibrálás
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30, #legvalószínűbb tokenek száma
        "temperature": 0.1, #generálás szabadságfoka
        "repetition_penalty": 1.03, #ismétlődések bűntetése
    },#kiegészítő paraméterek
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [16]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

#question kérdés, amire a RAG modell válaszolni fog
#llm, amit felhasználunk a modelhez
#vektorbázis, ami tartalmazza a tudást, mellyel a RAG modell válaszol

def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    #A knowledge index alapján RAG-gal válaszol
    # dokumentumok begyűjtése a retreiver-rel
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Opcionálisan rerank-olhatóak az eredmények
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Végső prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # válasz adása
    answer = llm(final_prompt)

    return answer, relevant_docs

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
